In [1]:
# !wget https://huggingface.co/datasets/mesolitica/semisupervised-abstractive-summarization-ms-news/resolve/main/populate-news.json.semisupervised

In [2]:
import json
import malaya
import numpy as np
from tqdm import tqdm
from unidecode import unidecode
import random
import re

months = {
    'january',
    'jan',
    'januari',
    'february',
    'feb',
    'februari',
    'march',
    'mac',
    'april',
    'apr',
    'may',
    'mei',
    'june',
    'jun',
    'july',
    'julai',
    'august',
    'ogos',
    'aug',
    'september',
    'sep',
    'october',
    'oktober',
    'oct',
    'november',
    'nov',
    'december',
    'disember',
    'dec',
    'utusan',
    'malaysiakini',
    'astroawani',
    'bernama',
    'com',
}

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

2023-07-25 12:49:16.041984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 12:49:16.117495: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 12:49:16.667440: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-25 12:49:16.667473: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [3]:
language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-ms-id')

In [4]:
before, after = [], []

count = 0
rejected = []
languages = []
accepted = []
para = []
malaysian_news = {
    'kosmo',
    'hmetro',
    'malaymail',
    'projekmm',
    'bharian',
    'utusan',
    'astroawani',
    'themalaysianinsight',
    'malaysiakini',
    'bernama'
}

def reject(data):
    if data['news'] in malaysian_news:
        return False
    if any([n in data['top-image'] for n in malaysian_news]):
        return False
    if any([n in data['url'] for n in malaysian_news]):
        return False
    if 'com.my' in data['top-image']:
        return False
    if data['language'] == 'malay':
        return False
    if 'Siaran Pers' in data['news']:
        return True
    if '.id' in data['news']:
        return True
    
    return True

filtered = []
with open('populate-news.json.semisupervised') as fopen:
    for l in tqdm(fopen):
        data = json.loads(l)
        text = re.sub(r'[ ]+', ' ', data['text']).strip()
        if not len(data['semisupervised-summaries']):
            continue
        
        if 'summary' in data:
            summary = data['summary']
        else:
            lens = [len(s) for s in data['semisupervised-summaries']]
            summary = data['semisupervised-summaries'][np.argmax(lens)]
            
        summary_text = re.sub(r'[ ]+', ' ', summary).strip()
        
        if 'kindly register' in text.lower() or 'disabled in your browser' in text.lower():
            continue
            
        if len(text.split()) < 50:
            continue
        
        if (len(summary_text.split()) / len(text.split())) < 0.5:
            continue
            
        langs = language_detection.predict([data['text'], summary])
        if not all([l == 'standard-malay' for l in langs]):
            continue
        
        filtered.append({
            'text': data['text'],
            'summary': summary,
        })

81717it [00:10, 7761.33it/s] 


In [5]:
len(filtered)

21457

In [6]:
filtered[1]

{'text': 'PRESIDEN Amerika Syarikat (AS), Donald Trump pada Jumaat mengarahkan sekatan terbaru ke atas Korea Utara ditarik balik, memberikan isyarat AS bersedia untuk meneruskan rundingan mengenai isu senjata nuklear dengan rejim itu, lapor agensi berita Yonhap.\n\nArahan itu dipercayai merujuk kepada sekatan oleh Jabatan Perbendaharaan AS ke atas dua syarikat perkapalan China yang dituduh membantu Korea Utara mengelak daripada sekatan AS.',
 'summary': 'Sekatan terbaru ke atas Korea Utara ditarik balik, memberikan isyarat AS bersedia untuk meneruskan rundingan. Arahan dipercayai merujuk kepada sekatan oleh Jabatan Perbendaharaan AS ke atas dua syarikat perkapalan China yang dituduh membantu Korea Utara mengelak daripada sekatan AS.'}

In [7]:
templates = [
    'paragraph `{t}`, summarize the paragraph',
    'paragraph `{t}`, ringkaskan paragraph',
    'ringkaskan teks `{t}`',
]

In [8]:
template_malaya = {
    'description': 'Template used by Malaya.',
    'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan, termasuk dengan input yang menyediakan konteks lanjut. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Input:\n{input}\n\n### Jawapan:\n',
    'prompt_no_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
    'response_split': '### Jawapan:',
}

In [12]:
import random

data = []
for d in filtered:
    try:
        t = d['text']
        a = d['summary']
        q = random.choice(templates).format(t = t)
        res = template_malaya["prompt_no_input"].format(instruction=q)
        res = f"{res}{a}"
        o = {
            'prompt_input': template_malaya["prompt_no_input"],
            'input': q,
            'output': a
        }
        data.append(o)

    except Exception as e:
        print(e)
        pass

In [13]:
data[0]

{'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
 'input': 'paragraph `POLIS akan memohon kepada Mahkamah Majistret di Johor Bahru hari ini untuk menyambung perintah tahanan reman ke atas lapan suspek berhubung kes pencemaran sisa kimia di Sungai Kim Kim, Pasir Gudang.\n\nKetua Polis Johor Mohd Khalil Kader Mohd berkata permohonan sambungan reman ke atas kesemua suspek ialah bagi membantu polis melengkapkan kertas siasatan.`, summarize the paragraph',
 'output': 'Polis Johor akan memohon untuk menyambung perintah tahanan reman terhadap lapan suspek. Ketua Polis Johor Mohd Khalil Kader Mohd berkata permohonan itu bagi membantu polis melengkapkan kertas siasatan.'}

In [14]:
with open('prepared-summarization.jsonl', 'w') as fopen:
    for l in data:
        fopen.write(f'{json.dumps(l)}\n')